In [1]:
import os, gams, pandas as pd, numpy as np, gams2numpy, shutil,pickle,pyDatabases
from pyDatabases import gpy, GpyDB
data = os.path.join(os.getcwd(), 'testdata')

*Load test databases:*

In [2]:
fs = [f"{data}\\test_size1000.gdx", f"{data}\\baselinerun.gdx"] # files
ws = gams.GamsWorkspace() 
g2np = gams2numpy.Gams2Numpy(ws.system_directory)
dbs = {'gms1': ws.add_database_from_gdx(fs[0]), 'gms2': ws.add_database_from_gdx(fs[1]),
       'gpy1': GpyDB(db=fs[0]), 'gpy2': GpyDB(db=fs[1])}

# gpy

*gpyDB* facilitates interaction with GAMS databases from Python using pandas and the Python API from GAMS. This notebook briefly goes through *gpy* class.

The supertype for symbols used in GAMS (akin to ```gams._GamsSymbol```). Instances of this class are initialized in one of three ways and always specifies four attributes:

```python
class gpy:
```  
* ```vals:``` Pandas representation of the symbol. The type of pandas representation depends on the ```self.type``` of the symbol. 
* ```name:``` Name of the symbol.  
* ```type:``` Subtype of symbol \{'set', 'subset', 'mapping', 'scalar_variable', 'scalar_parameter', 'variable', 'parameter'\}.
* ```text:``` description of symbol.

A few basic methods are defined for the class to help e.g. with various iteration schemes:
* ```self.__iter__```: Works on ```self.vals```. Thus, iteration automatically goes through pandas representation of the symbol.
* ```self.__len__```: Works on ```self.vals```. Thus, length looks at length of pandas representation.
* ```self.items()```: Works on ```self.__dict__```. 
* ```self.copy()```.

### 1: Subtypes
The pandas representation can ```pd.Index```, ```pd.MultiIndex```, ```pd.Series```, or simply a scalar (see variable ```DataBase.admissable_py_types```), depending on the subtype. In general, sets, subsets, and mappings are defined as pandas indices and parameters/variables are defined as pandas series. The differences between sets and subsets (both ```pd.Index```) and scalars/parameters/variables are more subtle. The function ```DataBase._type(symbol)``` returns the type.

#### ```gams.GamsSet```
The ```gams.GamsSet``` is split into three subtypes for the ```gpy_symbol```: \{'set','subset','mapping'\}. The type *set* is defined as a ```pd.Index``` where the *name* attribute corresponds to the *name* attribute of the ```gpy_symbol``` itself. Correspondingly, *subsets* are defined as ```pd.Index``` where the *name* attributes are not aligned:

In [3]:
db = dbs['gpy1']
setname, subset = 'i','subset'
db[setname].vals.name == setname, db[subset].vals.name == subset

(True, False)

*Mappings* are defined as multidimensional sets and stored as ```pd.MultiIndex``` instances. The *names* attribute include domains as strings referring to the *self.name* attributes of corresponding *sets*:

In [4]:
mapname = 'map'
db['map'].vals

MultiIndex([(1,   1),
            (1,   0),
            (1,   2),
            (1,   3),
            (1,   4),
            (1,   5),
            (1,   6),
            (1,   7),
            (1,   8),
            (1,   9),
            ...
            (1, 990),
            (1, 991),
            (1, 992),
            (1, 993),
            (1, 994),
            (1, 995),
            (1, 996),
            (1, 997),
            (1, 998),
            (1, 999)],
           names=['i', 'j'], length=1000)

#### ```gams.GamsVariable```
The ```gams.GamsVariable``` type is split into two subtypes for the ```gpy```: \{'scalar_variable','variable'\}. *Scalars* are defined as non-iterable objects e.g. integers, floats, or similar. *Variables* are stored as ```pd.Series``` defined over relevant indices.

In [5]:
var,scalar_var = 'var','scalar'
print("Variable:\n", db[var].vals,
      "\nScalar:", db[scalar_var].vals)

Variable:
 i  j  
1  1      10.0
   0      10.0
   2      10.0
   3      10.0
   4      10.0
          ... 
   995    10.0
   996    10.0
   997    10.0
   998    10.0
   999    10.0
Name: level, Length: 1000, dtype: object 
Scalar: 1.0


Importantly, the ```gpy*``` currently only stores the attribute *level*, where the records in ```gams.GamsVariable``` includes ('level','marginal','lower','upper','scale') attributes.

#### ```gams.GamsParameter```
The ```gams.GamsParameter``` type is split into two subtypes for the ```gpy```: \{'scalar_parameter','parameter'\}. Conventions are similar to variables.

In [6]:
param,param_scalar = 'param','pscalar'
print("Parameter:\n", db[param].vals,
      "\nScalar:", db[param_scalar].vals)

Parameter:
 i  j
1  1    20.0
   0    20.0
   2    20.0
   3    20.0
   4    20.0
   5    20.0
   6    20.0
   7    20.0
   8    20.0
   9    20.0
Name: value, dtype: object 
Scalar: 2.0


### 2: Initialization

```gpy``` can be initialized in three ways: *gpy*, *dict*, or *pandas-like* object (the ```gpy_db``` class builds methods on this initialization methods to add symbols from gdx and gmd databases). Initialization using *dicts* and *gpy* are fastest, but do not perform any check of validity; thus, they are less robust.

*Initialization with gpy:* The symbol copies the attributes from ```gpy.__dict__```; if kwargs are included, these overwrite existing attributes. This, for instance, initializes a copy of the symbol, and adjusts the *.text* attribute:

In [7]:
new_symbol = gpy(db['var'],**{'text': "this is the same variable, but with new text"})

*Initialization with dicts:* defines attributes for each (key,value) pair. Thus, this is identical to initializing with a ```gpy.__dict__```

In [8]:
new_symbol = gpy(new_symbol.__dict__)

*Initialization with pandas-like symbols:* The initialization depends on the type of symbol we wish to add. The following provides simple examples for each type:

*Variable/scalar_variables:* Keyword arguments are optional. Names can be added through the name of the series, or through kwargs.

In [9]:
var_from_pd = gpy(pd.Series([1], index = pd.Index(['i1'],name = 's'), name = 'variable_instance'),
                  **{'name': 'variable_instance',
                     'text': "some explanation here"})
var_scalar  = gpy(0, **{'name': 'scalar_variable_instance', 'text':"explanatory text"})

*Parameters/scalar_parameters:* As they are organized as variables/scalar_variables, the type has to be specified directly here.

In [10]:
par_from_pd = gpy(var_from_pd.vals.rename('parameter_instance'), **{'type': 'paramater'})
par_scalar = gpy(0, **{'name': 'scalar_parameter_instance', 'type': 'scalar_parameter', 'text': "..."})

*Sets:* Mappings are ```pd.MultiIndex```, sets/subsets are ```pd.Index```, distinguished by alignment/misalignment of names (as explained above).

In [11]:
s = gpy(pd.Index(range(10),name='s'), **{'name':'s'}) # set
ss = gpy(pd.Index(range(5), name='s'), **{'name':'ss'}) # subset
m = gpy(pd.MultiIndex.from_tuples([(0,1)], names = ['s','i']),name='mapping_instance') # mapping
(s.type, ss.type, m.type) # print types

('set', 'subset', 'mapping')

### 3: Properties

*gpy*s come with a couple of simple build-in properties:
* ```self.index```: Returns pandas index. If the symbol is a set it returns *self.vals*. If the symbol is a variable/parameter it returns the domains. otherwise it returns None.
* ```self.domains```: Returns list of domains as strings. If *self.index* is None it returns empty list.
* ```self.df```: Returns pd.DataFrame used in gams2numpy to write to gams-type databases. Only works for gpy instances with ```self.vals``` is a pd.Series instance.

Examples:

```python
self.index
```

In [12]:
var_from_pd.index

Index(['i1'], dtype='object', name='s')

In [13]:
m.index

MultiIndex([(0, 1)],
           names=['s', 'i'])

In [14]:
var_scalar == None

False

```python 
self.domains
```

In [15]:
var_from_pd.domains

FrozenList(['s'])

In [16]:
m.domains

FrozenList(['s', 'i'])

In [17]:
var_scalar.domains

[]

```python 
self.df
```

In [18]:
var_from_pd.df

,level
s,
i1,1


In [19]:
par_from_pd.df

,value
s,
i1,1


In [20]:
try:
    m.df
except:
    print('Mapping is not defined as pd.Series; cannot convert to dataframe.')

Mapping is not defined as pd.Series; cannot convert to dataframe.
